In [6]:
import pandas as pd
import numpy as np
from line_profiler import LineProfiler

%load_ext Cython
%load_ext line_profiler

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [7]:
!pip install line_profiler --user

In [8]:
# https://www.kaggle.com/apratim87/housingdata
data = pd.read_csv('housingdata.csv', header=None)

In [9]:
data = (data - data.mean()) / data.std()

In [10]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.419367,0.284548,-1.286636,-0.272329,-0.144075,0.413263,-0.119895,0.140075,-0.981871,-0.665949,-1.457558,0.440616,-1.074499,0.159528
1,-0.416927,-0.487240,-0.592794,-0.272329,-0.739530,0.194082,0.366803,0.556609,-0.867024,-0.986353,-0.302794,0.440616,-0.491953,-0.101424
2,-0.416929,-0.487240,-0.592794,-0.272329,-0.739530,1.281446,-0.265549,0.556609,-0.867024,-0.986353,-0.302794,0.396035,-1.207532,1.322937
3,-0.416338,-0.487240,-1.305586,-0.272329,-0.834458,1.015298,-0.809088,1.076671,-0.752178,-1.105022,0.112920,0.415751,-1.360171,1.181589
4,-0.412074,-0.487240,-1.305586,-0.272329,-0.834458,1.227362,-0.510674,1.076671,-0.752178,-1.105022,0.112920,0.440616,-1.025487,1.486032


In [11]:
def profile_print(func_to_call, *args):
    profiler = LineProfiler()
    profiler.add_function(func_to_call)
    profiler.runcall(func_to_call, *args)
    profiler.print_stats()

In [12]:
X = data.values[:, :-1]
y = data.values[:, -1:]

У нас есть выборка $X \in R^{n\times d}, y\in R^n$, состоящая из $n$ объектов. Каждый из $n$ объектов описывается вектором из $d$ признаков (строка матрицы $X$) и для каждого объекта мы знаем значение целевой переменной $y$. В данной задаче по некоторым параметрам мы хотим научиться предсказывать стоимость квадратного метра жилья.


Предположим, что для объекта $i$ мы можем описать $y_i$ линейной комбинацией $x_i$ с некоторыми весами $w$, где w, x_i - вектора размера $d \times 1$, а $y$ - вещественное число.
$$y_i \sim <w,x_i>$$


Методом наименьших квадратов найдем веса $w$.
$$ J(w) = \frac{1}{n} \sum_{i=1}^n (<w,x_i> - y_i)^2$$
$$ J(w) \rightarrow \min_w $$

Минимум данного функционала будем искать методом градиентного спуска (те будет идти по направлению противоположному градиенту):
$$ w_j = w_j - \alpha \frac{\partial}{\partial w_j}J(w) $$
$$ \frac{\partial}{\partial w_j}J(w)  = \frac{2}{n}  \sum_{i=1}^n (<w, x_i> - y_i)\cdot x_i^j $$

In [13]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr = lr.fit(X, y)
print ((y - lr.predict(X)) ** 2).mean()

0.258844771986


In [14]:
lr.coef_

array([[-0.10101708,  0.1177152 ,  0.0153352 ,  0.07419883, -0.22384803,
         0.29105647,  0.00211864, -0.33783635,  0.28974905, -0.22603168,
        -0.22427123,  0.09243223, -0.40744693]])

In [15]:
N_ITER = 300

In [11]:
def gradient_decent_python_naive(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        for f in xrange(n_features):
            gradient = 0
            for obj in xrange(n_objects):
                gradient += ((X[obj, :] * w_old).sum() - y[obj, 0]) * X[obj, f] 
            gradient = gradient * 2 / n_objects
            w[f] = w_old[f] - alpha * gradient
    return w.reshape(-1, 1)

In [12]:
w = gradient_decent_python_naive(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258846303836


In [13]:
%%timeit
gradient_decent_python_naive(X, y, n_iter=N_ITER)

1 loop, best of 3: 1min 3s per loop


In [14]:
profile_print(gradient_decent_python_naive, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 86.5583 s
File: <ipython-input-11-ee2da1012c6c>
Function: gradient_decent_python_naive at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def gradient_decent_python_naive(X, y, n_iter=100, alpha=0.1):
     2         1            7      7.0      0.0      n_objects, n_features = X.shape
     3         1           29     29.0      0.0      w = np.random.rand(n_features)
     4         1           10     10.0      0.0      w_old = np.random.rand(n_features)
     5       301          596      2.0      0.0      for iteration in xrange(n_iter):
     6       300         6693     22.3      0.0          np.copyto(w_old, w)
     7      4200        10927      2.6      0.0          for f in xrange(n_features):
     8      3900         6700      1.7      0.0              gradient = 0
     9   1977300      5052402      2.6      5.8              for obj in xrange(n_objects):
    10   1973400     81

In [15]:
#  поменяли порядок циклов по объектам и признакам

def gradient_decent_python(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            for f in xrange(n_features):
                gradient[f] += ((X[obj, :] * w_old).sum() - y[obj, 0]) * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [16]:
w = gradient_decent_python(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258850940896


In [17]:
%%timeit
gradient_decent_python(X, y, n_iter=N_ITER)

1 loop, best of 3: 1min 4s per loop


In [18]:
profile_print(gradient_decent_python, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 85.6283 s
File: <ipython-input-15-9de9ecf96cb1>
Function: gradient_decent_python at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_python(X, y, n_iter=100, alpha=0.1):
     4         1            8      8.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           30     30.0      0.0      w = np.random.rand(n_features)
     6         1           12     12.0      0.0      w_old = np.random.rand(n_features)
     7       301          814      2.7      0.0      for iteration in xrange(n_iter):
     8       300         3582     11.9      0.0          np.copyto(w_old, w)
     9       300         3843     12.8      0.0          gradient = np.zeros(n_features)
    10    152100       281268      1.8      0.3          for obj in xrange(n_objects):
    11   2125200      5565994      2.6      6.5              for f in xrange(n_features):
    1

In [19]:
#  вынесем подсчет ошибки

def gradient_decent_python_faster(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            diff = diff_count(X, obj, w, y)
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

def diff_count(X, obj, w, y):
    return (X[obj, :] * w).sum() - y[obj, 0]

In [24]:
%lprun -f gradient_decent_python_faster -f diff_count gradient_decent_python_faster(X, y, n_iter=N_ITER)

In [25]:
w = gradient_decent_python_faster(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258864201293


In [26]:
%%timeit
gradient_decent_python_faster(X, y, n_iter=N_ITER)

1 loop, best of 3: 8.42 s per loop


In [27]:
profile_print(gradient_decent_python_faster, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 20.6977 s
File: <ipython-input-19-c08a554b2762>
Function: gradient_decent_python_faster at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_python_faster(X, y, n_iter=100, alpha=0.1):
     4         1            8      8.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           32     32.0      0.0      w = np.random.rand(n_features)
     6         1           12     12.0      0.0      w_old = np.random.rand(n_features)
     7       301          799      2.7      0.0      for iteration in xrange(n_iter):
     8       300         3452     11.5      0.0          np.copyto(w_old, w)
     9       300         3833     12.8      0.0          gradient = np.zeros(n_features)
    10    152100       274667      1.8      1.3          for obj in xrange(n_objects):
    11    151800      6394897     42.1     30.9              diff = diff_count(X,

In [16]:
from numba import jit

In [17]:
# numba !!!

@jit(nopython=True)
def gradient_decent_numba(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        for f in xrange(n_features):  # changed
            w_old[f] = w[f]
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            diff = (X[obj, :] * w).sum() - y[obj, 0]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [18]:
w = gradient_decent_numba(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258849507918


In [19]:
%%timeit
gradient_decent_numba(X, y, n_iter=N_ITER)

10 loops, best of 3: 52.9 ms per loop


In [20]:
# уберем цикл по объектам

def gradient_decent_numpy(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    w_old = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = np.zeros((n_features, 1))  # chaned
        diff = X.dot(w) - y
        for f in xrange(n_features):
            gradient[f, 0] = (diff * X[:, f:f+1]).sum()
        gradient = gradient * 2 / n_objects
        w = w - alpha * gradient
    return w  # changed

In [24]:
w = gradient_decent_numpy(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258852947222


In [25]:
%%timeit
gradient_decent_numpy(X, y, n_iter=N_ITER)

10 loops, best of 3: 135 ms per loop


In [26]:
profile_print(gradient_decent_numpy, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 0.225635 s
File: <ipython-input-20-7d6ed977bab5>
Function: gradient_decent_numpy at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_numpy(X, y, n_iter=100, alpha=0.1):
     4         1            9      9.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           52     52.0      0.0      w = np.random.rand(n_features, 1)  # changed
     6         1           18     18.0      0.0      w_old = np.random.rand(n_features, 1)  # changed
     7       301          820      2.7      0.4      for iteration in xrange(n_iter):
     8       300         4242     14.1      1.9          gradient = np.zeros((n_features, 1))  # chaned
     9       300        17431     58.1      7.7          diff = X.dot(w) - y
    10      4200        16998      4.0      7.5          for f in xrange(n_features):
    11      3900       165636     42.5     73.4       

In [46]:
# уберем еще и цикл по признакам

def gradient_decent_numpy_faster(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        diff = X.dot(w) - y
        #for f in xrange(n_features):
            #gradient[f, 0] = (diff * X[:, f:f+1]).sum()
        gradient = (diff * X).sum(axis=0)
        gradient = gradient.reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [47]:
w = gradient_decent_numpy_faster(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258846100011


In [48]:
%%timeit
gradient_decent_numpy_faster(X, y, n_iter=N_ITER)

10 loops, best of 3: 35.9 ms per loop


In [49]:
profile_print(gradient_decent_numpy_faster, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 0.05318 s
File: <ipython-input-46-b8203c9009d8>
Function: gradient_decent_numpy_faster at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_numpy_faster(X, y, n_iter=100, alpha=0.1):
     4         1            8      8.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           30     30.0      0.1      w = np.random.rand(n_features, 1)  # changed
     6       301          848      2.8      1.6      for iteration in xrange(n_iter):
     7       300         9224     30.7     17.3          diff = X.dot(w) - y
     8       300        25020     83.4     47.0          gradient = (diff * X).sum(axis=0)
     9       300         2404      8.0      4.5          gradient = gradient.reshape(-1, 1)
    10       300         9168     30.6     17.2          gradient = gradient * 2 / n_objects
    11       300         6476     21.6     12.2          

In [50]:
@jit(nopython=True)
def gradient_decent_numpy_faster_numba(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [52]:
# самая большая проблема - sum c axis

w = gradient_decent_numpy_faster_numba(X, y)
print ((y - X.dot(w)) ** 2).mean()

UntypedAttributeError: Caused By:
Traceback (most recent call last):
  File "/home/ekaterina/.local/lib/python2.7/site-packages/numba/compiler.py", line 238, in run
    stage()
  File "/home/ekaterina/.local/lib/python2.7/site-packages/numba/compiler.py", line 452, in stage_nopython_frontend
    self.locals)
  File "/home/ekaterina/.local/lib/python2.7/site-packages/numba/compiler.py", line 865, in type_inference_stage
    infer.propagate()
  File "/home/ekaterina/.local/lib/python2.7/site-packages/numba/typeinfer.py", line 844, in propagate
    raise errors[0]
UntypedAttributeError: Unknown attribute 'dot' of type array(float64, 2d, F)
File "<ipython-input-50-a8ae7606de61>", line 6
[1] During: typing of get attribute at <ipython-input-50-a8ae7606de61> (6)

Failed at nopython (nopython frontend)
Unknown attribute 'dot' of type array(float64, 2d, F)
File "<ipython-input-50-a8ae7606de61>", line 6
[1] During: typing of get attribute at <ipython-input-50-a8ae7606de61> (6)

In [53]:
%%cython
import numpy as np
cimport numpy as np

def gradient_decent_numpy_faster_cython(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [54]:
w = gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258845458115


In [55]:
%%timeit
gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)

10 loops, best of 3: 37.4 ms per loop


In [56]:
# аннтоции 

In [57]:
%%cython -a
import numpy as np
cimport numpy as np

def gradient_decent_numpy_faster_cython(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [58]:
# добавим типы

In [59]:
%%cython -a
import numpy as np
cimport numpy as np

cpdef gradient_decent_numpy_faster_cython(np.ndarray[np.float64_t, ndim=2] X, 
                                          np.ndarray[np.float64_t, ndim=2] y, 
                                          int n_iter=100, 
                                          np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.ndarray[np.float64_t, ndim=2] w;
    cdef np.ndarray[np.float64_t, ndim=2] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [60]:
w = gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258870109448


In [61]:
%%timeit
gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)

10 loops, best of 3: 39.7 ms per loop


In [62]:
#  вернемся к нашему хорошему коду на питоне и добавим типы

In [63]:
%%cython -a
import numpy as np
cimport numpy as np


cpdef gradient_decent_python_faster_cython_v0(np.ndarray[np.float64_t, ndim=2] X, 
                                              np.ndarray[np.float64_t, ndim=2] y, 
                                              int n_iter=100, 
                                              np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.ndarray[np.float64_t, ndim=1] w;
    cdef np.ndarray[np.float64_t, ndim=1] w_old;
    cdef np.ndarray[np.float64_t, ndim=1] gradient;
    cdef np.float64_t diff;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            diff = (X[obj, :] * w).sum() - y[obj, 0]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [64]:
w = gradient_decent_python_faster_cython_v0(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258845417426


In [65]:
%%timeit
res = gradient_decent_python_faster_cython_v0(X, y, n_iter=N_ITER)

1 loop, best of 3: 4.4 s per loop


In [66]:
# заменим код, где много вызовов к python (отмечены желтым)

In [67]:
%%cython -a
import numpy as np
cimport numpy as np


cpdef gradient_decent_python_faster_cython_v1(np.ndarray[np.float64_t, ndim=2] X, 
                                              np.ndarray[np.float64_t, ndim=2] y, 
                                              int n_iter=100, 
                                              np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.float64_t diff
    cdef np.ndarray[np.float64_t, ndim=1] w;
    cdef np.ndarray[np.float64_t, ndim=1] w_old;
    cdef np.ndarray[np.float64_t, ndim=1] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    gradient = np.zeros(n_features)
    for iteration in xrange(n_iter):
        for f in xrange(n_features):
            gradient[f] = 0
#             w_old[f] = w[f]
        for obj in xrange(n_objects):
            diff = - y[obj, 0]
            for f in xrange(n_features):
                diff += X[obj, f] * w[f]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        for f in xrange(n_features):
            gradient[f] = gradient[f] * 2 / n_objects
            w[f] = w[f] - alpha * gradient[f]
    return w.reshape(-1, 1)

In [68]:
w = gradient_decent_python_faster_cython_v1(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258861481982


In [69]:
%%timeit
res = gradient_decent_python_faster_cython_v1(X, y, n_iter=N_ITER)

10 loops, best of 3: 26.3 ms per loop


In [70]:
# добавим декораторы

In [71]:
%%cython -a
import numpy as np
cimport numpy as np
import cython


@cython.boundscheck(False)
@cython.cdivision(True)
cpdef gradient_decent_python_faster_cython_v2(np.ndarray[np.float64_t, ndim=2] X, 
                                              np.ndarray[np.float64_t, ndim=2] y, 
                                              int n_iter=100, 
                                              np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.float64_t diff
    cdef np.ndarray[np.float64_t, ndim=1] w;
    cdef np.ndarray[np.float64_t, ndim=1] w_old;
    cdef np.ndarray[np.float64_t, ndim=1] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    gradient = np.zeros(n_features)
    for iteration in xrange(n_iter):
        for f in xrange(n_features):
            gradient[f] = 0
            w_old[f] = w[f]
        for obj in xrange(n_objects):
            diff = - y[obj, 0]
            for f in xrange(n_features):
                diff += X[obj, f] * w[f]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        for f in xrange(n_features):
            gradient[f] = gradient[f] * 2 / n_objects
            w[f] = w_old[f] - alpha * gradient[f]
    return w.reshape(-1, 1)

In [72]:
w = gradient_decent_python_faster_cython_v2(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258853138266


In [73]:
%%timeit
res = gradient_decent_python_faster_cython_v2(X, y, n_iter=N_ITER)

100 loops, best of 3: 16.5 ms per loop
